# How to exploit the month-end flow effect
Fund managers report their holdings every month. They don’t want to tell investors that they lost money the latest meme stock. So they will sell the meme stocks and buy higher quality assets, like bonds.

We might be able to take advantage of this effect by buying bonds toward the end of the month and selling them at the beginning.

In [1]:
# import the libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import vectorbt as vbt

In [2]:
# Load the data
tlt = vbt.YFData.download(
    "TLT", 
    start="2004-01-01"
).get("Close").to_frame()
close = tlt.Close

In [3]:
# Building the trading signals
short_entries = pd.DataFrame.vbt.signals.empty_like(close)
short_exits = pd.DataFrame.vbt.signals.empty_like(close)
long_entries = pd.DataFrame.vbt.signals.empty_like(close)
long_exits = pd.DataFrame.vbt.signals.empty_like(close)

In [4]:
# Get short on the first day of the new month
short_entry_mask = ~tlt.index.tz_convert(None).to_period("M").duplicated()
short_entries.iloc[short_entry_mask] = True

# Exit the short five days later
short_exit_mask = short_entries.shift(5).fillna(False)
short_exits.iloc[short_exit_mask] = True

# Get long 7 days prior to the end of the month
long_entry_mask = short_entries.shift(-7).fillna(False)
long_entries.iloc[long_entry_mask] = True

# Buy back on the last day of the month
long_exit_mask = short_entries.shift(-1).fillna(False)
long_exits.iloc[long_exit_mask] = True

In [8]:
short_entries

Date
2003-12-31 05:00:00+00:00     True
2004-01-02 05:00:00+00:00     True
2004-01-05 05:00:00+00:00    False
2004-01-06 05:00:00+00:00    False
2004-01-07 05:00:00+00:00    False
                             ...  
2024-02-07 05:00:00+00:00    False
2024-02-08 05:00:00+00:00    False
2024-02-09 05:00:00+00:00    False
2024-02-12 05:00:00+00:00    False
2024-02-13 05:00:00+00:00    False
Name: Close, Length: 5064, dtype: bool

In [5]:
# Run the backtest
pf =  vbt.Portfolio.from_signals(
    close=close,
    entries=long_entries,
    exits=long_exits,
    short_entries=short_entries,
    short_exits=short_exits,
    freq="1d"
)

pf.stats()

Start                         2003-12-31 05:00:00+00:00
End                           2024-02-13 05:00:00+00:00
Period                               5064 days 00:00:00
Start Value                                       100.0
End Value                                    620.623016
Total Return [%]                             520.623016
Benchmark Return [%]                          105.98706
Max Gross Exposure [%]                            100.0
Total Fees Paid                                     0.0
Max Drawdown [%]                              21.079168
Max Drawdown Duration                 370 days 00:00:00
Total Trades                                        483
Total Closed Trades                                 483
Total Open Trades                                     0
Open Trade PnL                                      0.0
Win Rate [%]                                  59.213251
Best Trade [%]                                 6.465247
Worst Trade [%]                              -11

In [7]:
# plot the backtest
pf.plot()

FigureWidget({
    'data': [{'legendgroup': '0',
              'line': {'color': '#1f77b4'},
              'name': 'Close',
              'showlegend': True,
              'type': 'scatter',
              'uid': '48fedf0c-d9dc-4314-b352-8d33d1f98a13',
              'x': array([datetime.datetime(2003, 12, 31, 5, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2004, 1, 2, 5, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2004, 1, 5, 5, 0, tzinfo=datetime.timezone.utc), ...,
                          datetime.datetime(2024, 2, 9, 5, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 2, 12, 5, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 2, 13, 5, 0, tzinfo=datetime.timezone.utc)],
                         dtype=object),
              'xaxis': 'x',
              'y': array([44.83291245, 44.30302048, 44.18754959, ..., 93.84999847, 93.95999908,
                